In [1]:
# theano imports
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
# from theano.tensor.signal.downsample import max_pool_2d
from theano.tensor.signal.pool import pool_2d as max_pool_2d

# other imports
from sklearn.cross_validation import train_test_split
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read our data in 
with open('conv_data/5_tensor.p', 'rb') as f:
    loaded_data = pickle.load(f)

labels = pd.read_csv('one_hot_labels.csv')

In [3]:
one_hot = labels.values

In [4]:
xTrain, xTest, yTrain, yTest = train_test_split(loaded_data, one_hot)

In [5]:
xTrain = xTrain.reshape(-1, 1, 5, 21)
xTest = xTest.reshape(-1, 1, 5, 21)

In [6]:
print xTrain.shape, xTest.shape, yTrain.shape, yTest.shape

(101110, 1, 5, 21) (33704, 1, 5, 21) (101110, 6) (33704, 6)


In [7]:
srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def glorot_init_weights(shape):
    (h, w) = shape
    normalizer = 2.0 * (6**0.5) / ((h + w)**0.5) * 0.1  #factors: correct for uni[0,1], glo, glo, softmax deriv
    return theano.shared(floatX((np.random.random_sample(shape) - 0.5) * normalizer))

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def activ(x, f=None):
    if f == 'tan':
        return T.tanh(x)
    else:
        return T.nnet.sigmoid(x)

def dropout(X, p=0.0):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X
    
def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    
    return updates

def conv_model(X, wc, wh=None, p_drop_conv=0.0, p_drop_hidden=0.0):
    l_1 = T.nnet.relu(conv2d(X, wc, border_mode='valid'))
    l_1 = l_1.reshape((-1, 256))
#     l_1 = dropout(l_1, p_drop_conv)

#     l_2 = rectify(T.dot(layer_1, wh))
#     l_2 = dropout(layer_2, p_drop_hidden)
    
    return l_1

### Consider adding some sort of adaptive learning rate, for better weight updates

In [8]:
X = T.ftensor4()# dims (mbs, n_channels, n_rows, n_cols)
Y = T.fmatrix() # dims (mbs, o_depth)

mbs = 128 # mini-batch size
n_channels, n_rows, n_cols = 1, 5, 21 # define protein "image" dimensions
n_conv = 256 # features out of convolution
h_depth = 600 # hidden layer depth
o_depth = 6 # output depth

# initialize weights for conv layer
W_c = init_weights((n_conv, n_channels, n_rows, n_cols))

# initial weight matrices
W_uh = np.asarray(np.random.randn(n_conv, h_depth) * 0.001, dtype = theano.config.floatX)
W_hh = np.asarray(np.random.randn(h_depth, h_depth) * 0.001, dtype = theano.config.floatX)
W_hy = np.asarray(np.random.randn(h_depth, o_depth) * 0.001, dtype = theano.config.floatX)
b_hh = np.zeros((mbs, h_depth), dtype=theano.config.floatX)
b_hy = np.zeros((mbs, o_depth), dtype=theano.config.floatX)

W_uh = theano.shared(W_uh, 'W_uh')
W_hh = theano.shared(W_hh, 'W_hh')
W_hy = theano.shared(W_hy, 'W_hy')
b_hh = theano.shared(b_hh, 'b_hh')
b_hy = theano.shared(b_hy, 'b_hy')

# define new matrices 
Wr_uh = np.asarray(np.random.randn(n_conv, h_depth) * 0.001, dtype = theano.config.floatX)
Wr_hh = np.asarray(np.random.randn(h_depth, h_depth) * 0.001, dtype = theano.config.floatX)
Wz_uh = np.asarray(np.random.randn(n_conv, h_depth) * 0.001, dtype = theano.config.floatX)
Wz_hh = np.asarray(np.random.randn(h_depth, h_depth) * 0.001, dtype = theano.config.floatX)

Wr_uh = theano.shared(Wr_uh, 'Wr_uh')
Wr_hh = theano.shared(Wr_hh, 'Wr_hh')
Wz_uh = theano.shared(Wz_uh, 'Wz_uh')
Wz_hh = theano.shared(Wz_hh, 'Wz_hh')

h0_tm1 = theano.shared(np.zeros((mbs, h_depth), dtype=theano.config.floatX))

# have conv_model outside of recurrent fn
u = conv_model(X, W_c)

params = [W_c, W_hh, W_uh, W_hy, Wr_uh, Wr_hh, Wz_uh, Wz_hh, b_hh, b_hy]

def recurrent_fn(u_t, h_tm1, W_hh, W_uh, W_hy, Wr_uh, Wr_hh, Wz_uh, Wz_hh, b_hh, b_hy):
    
    r_t = activ(T.dot(u_t, Wr_uh) + T.dot(h_tm1, Wr_hh))
    z_t = activ(T.dot(u_t, Wz_uh) + T.dot(h_tm1, Wz_hh))
    
    h_tilda = activ(T.dot(u_t, W_uh) + r_t*T.dot(h_tm1, W_hh), f='tan')
    
    h_t = ((1 - z_t)*h_tm1) + (z_t*h_tilda) + b_hh
    
    # adding in softmax, for one hot coding 
    y_t = T.nnet.softmax(T.dot(h_t, W_hy) + b_hy)
    
    return h_t, y_t

[h, y], _ = theano.scan(recurrent_fn, 
                       sequences = u,
                       outputs_info = [h0_tm1, None],
                       non_sequences = [W_hh, W_uh, W_hy, Wr_uh, Wr_hh, Wz_uh, Wz_hh, b_hh, b_hy])


# for some reason, y is a (128, 128, 6) tensor

cost = T.mean((Y - y)**2)
# cost = T.mean(T.nnet.categorical_crossentropy(y, Y))

update = RMSprop(cost, params) #lr=1e-7

y_x = T.argmax(y, axis=1)

train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)

# number of training iterations to perform
n_train = 101

costs = []

# performing our training
for i in xrange(n_train):
    for start, end in zip(xrange(0, len(xTrain), mbs), 
                          xrange(mbs, len(xTrain), mbs)):
        cost = train(xTrain[start:end], yTrain[start:end])=
        print i, 'test:', np.mean(np.argmax(yTest, axis=1) == predict(xTest))
#         print i, 'training:', np.mean(np.argmax(yTrain, axis=1) == predict(xTrain))
    
    costs.append(cost)
    print i, np.mean(np.argmax(yTest, axis=1) == predict(xTest))

8.02777778833
(128, 128, 6)


MissingInputError: ("An input of the graph, used to compute ConvOp{('imshp', (None, None, None)),('kshp', (None, None)),('nkern', None),('bsize', None),('dx', 1),('dy', 1),('out_mode', 'valid'),('unroll_batch', None),('unroll_kern', None),('unroll_patch', True),('imshp_logical', (None, None, None)),('kshp_logical', (None, None)),('kshp_logical_top_aligned', True)}(<TensorType(float32, 4D)>, <TensorType(float32, 4D)>), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.", <TensorType(float32, 4D)>)

In [ ]:
# AHA
# for some reason y from the scan function is dimensions (128, 128, 6)